<a href="https://colab.research.google.com/github/mariaR5/newspaper-info-extractor/blob/main/Newspaper_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import google.generativeai as genai

In [ ]:
from pathlib import Path

In [ ]:
from google.colab import userdata

In [ ]:
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)

Printing the different gemini models available

In [ ]:
for model in genai.list_models():
  if "generateContent" in model.supported_generation_methods:
    print(model.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924


In [ ]:
generation_config = {
  "temperature": 0,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

In [ ]:
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE",
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE",
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE",
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE",
  },
]

In [ ]:
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config=generation_config,
    safety_settings=safety_settings
)

In [ ]:
def img_format(img_path):
  img = Path(img_path)

  if not img.exists():
    raise FileNotFoundError(f"Could not find {img}")

  img_parts = [
      {
          "mime_type" : "image/png",
          "data" : img.read_bytes()
      }
  ]

  return img_parts

In [ ]:
def gemini_output(sys_prompt, user_prompt, img_path):
  img_info = img_format(img_path)
  input_prompt = [sys_prompt, user_prompt, img_info[0]]
  response = model.generate_content(input_prompt)
  return response.text

In [ ]:
sys_prompt = """You are capable of categorizing articles of a newspaper into distinct topics while efficiently separating headlines, publishers, content, and metadata such as locations of occurrences.
Implement advanced natural language processing techniques to ensure accurate identification and extraction, facilitating seamless organization and analysis of news content.
Input images of newspaper clipping will be provided to you.
Your task is to respond to questions based on the content of the input image."""


img_path = "Punjab_Times.jpg"


user_prompt = """analyse the newspaper and divide them into indivitual aritcles
and convert the data into json format using appropriate json tags(headline, location, summarised content) as required for each article"""

In [ ]:
output = gemini_output(sys_prompt, user_prompt, img_path)

In [ ]:
import json
from pprint import pprint

# Assuming 'output' contains your JSON object
pprint(output)


('```json\n'
 '[\n'
 '  {\n'
 '    "headline": "Modi Govt 3.0 holds first Cabinet meet",\n'
 '    "location": null,\n'
 '    "summarised_content": "The first Cabinet meet of the ruling NDA\'s third '
 'consecutive term focused on providing assistance to urban and rural houses '
 'under the Pradhan Mantri Awas Yojana since 2015-16. The government has been '
 'implementing Pradhan Mantri Awas Yojana since 2015-16 to provide basic '
 'amenities of houses with construction of toilets, tap connections, and '
 'stable and for rural houses, schemes for the eligible families under the '
 'housing for all (HFA) scheme were completed in the last 10 years under '
 'PMAY.  The basic amenities with the other provided schemes are in last 10 '
 'years provided such as functional household tap connection, functional '
 'toilet, electricity connection through convergence with governments and '
 'state governments."\n'
 '  },\n'
 '  {\n'
 '    "headline": "CM Pays Obeisance at Gurudwara in Mohali",\n'
 

In [ ]:
from IPython.display import Markdown
Markdown(output)

```json
[
  {
    "headline": "Modi Govt 3.0 holds first Cabinet meet",
    "location": null,
    "summarised_content": "The first Cabinet meet of the ruling NDA's third consecutive term focused on providing assistance to urban and rural houses under the Pradhan Mantri Awas Yojana since 2015-16. The government has been implementing Pradhan Mantri Awas Yojana since 2015-16 to provide basic amenities of houses with construction of toilets, tap connections, and stable and for rural houses, schemes for the eligible families under the housing for all (HFA) scheme were completed in the last 10 years under PMAY.  The basic amenities with the other provided schemes are in last 10 years provided such as functional household tap connection, functional toilet, electricity connection through convergence with governments and state governments."
  },
  {
    "headline": "CM Pays Obeisance at Gurudwara in Mohali",
    "location": "Mohali",
    "summarised_content": "Punjab Chief Minister Bhagwant Mann paid obeisance at Gurudwara Shaheedan, interacting with the media, said that the AAP has performed well in the Lok Sabha polls as the share vote has more than doubled compared to the previous elections. He said that more than 20% during current vote share Congress has secured three seats and BJP has been decimated. He said that the entire vote share of time has been divided and BSP has been wiped out. However, he said that the AAP is working hard and improve its working and try to improve it. Bhagwant Singh Mann said that the land code was too long of development. The DS core has prepared the released 91 crore pending payment for the Matric scholarship scheme as said that 90% of the students seeking immigrant to Canada. He state that art of the education is being imparted to students."
  },
  {
    "headline": "PM Modi assumes office; signs first file about release of PM Kisan Nidhi funds",
    "location": "New Delhi",
    "summarised_content": "Prime Minister Narendra Modi on Monday assumed office committed to a government working for the poor and the common man. The PM took the ongoing portal and moved to small and marginal farmers. The PM signed his first file authorizing the installment of 9.5 crore farmers through direct benefit transfer to the eligible families out of the increase in the number of eligible families."
  },
  {
    "headline": "Pak-backed TRF claims responsibility for Reasi bus terror attack",
    "location": "Reasi",
    "summarised_content": "The Resistance Front (TRF), an offshoot of the terrorist organisation The LeT, claimed responsibility for the terror attack on a bus carrying pilgrims in a district on Sunday. The bus attack claimed the lives of 7 tourists and wounded 33 others. The TRF claimed that the attack was carried out by its local men. The LeT and its offshoots have been carrying out such attacks on tourists and locals. The LeT is a terrorist organisation that has been designated as a terrorist organisation by the United Nations. The TRF is a new outfit that was founded in 2019 after the abrogation of Article 370."
  },
  {
    "headline": "J&K LG Manoj Sinha announces Rs 10 lakh ex-gratia each for families of victims",
    "location": "Jammu and Kashmir",
    "summarised_content": "Lieutenant Governor Manoj Sinha on Monday announced an ex-gratia of Rs 50 lakh each to the injured and Rs 10 lakh to each of the control room has been created by the district administration to provide necessary assistance. A joint security force temporary HQ and a control room has been set up in progress to neutralise the perpetrators of the operation is in progress. A search operation by J&K Police, Army, and CRPF is underway, as per a statement issued by the LG's office."
  },
  {
    "headline": "CBSE issues warning against fake syllabus, sample question papers",
    "location": null,
    "summarised_content": "The Central Board of Secondary Education (CBSE) on Monday issued a warning against misleading advertisements and fake syllabus, sample question papers, and sample question papers for 2023-24 session. The Board has urged parents and students to come to our official portal. It has come to our notice that certain online portals and websites are circulating outdated links, activities, CBSE resources, and fake news. These links and resources are being circulated for the session 2022-23. In the interest of the students, the CBSE has emphasized that information from unauthorized sources can mislead students, parents, and other stakeholders, including schools."
  },
  {
    "headline": "Jalandhar West By-Election On July 10: Sibin C",
    "location": "Jalandhar",
    "summarised_content": "Punjab Chief Electoral Officer Sibin C has informed that voting for the Jalandhar West by-election will be held on July 10. The Election Commission of India has released the election program in this regard. Filing nominations will be held for Friday and the last date (Friday) for the last date will be on June 21. The nomination papers will be scrutinized on June 24 (Monday), while the last date (Wednesday) will be on June 26. Voting will be held on that day as per agreement."
  },
  {
    "headline": "Himachal Pradesh to release water to Delhi as per agreement: CM Sukhu",
    "location": "Delhi",
    "summarised_content": "Chief Minister Sukhvinder Singh Sukhu on Monday said that the Himachal Pradesh government is ready to release water to Delhi as per the agreement."
  },
  {
    "headline": "In memory of Baba Nihal Singh ji, annual Shaheedi Jor Mela at Gurudwara Saheeban village Talhan from 14 June",
    "location": "Talhan",
    "summarised_content": "The 73rd Annual Shaheedi Jor Mela is starting from 14 June at Gurudwara Saheeban village Talhan. The Gurudwara Saheeban is located in village Talhan and is just 11 km from Jalandhar and will continue till June 16. The Gurudwara Saheeban will receive Jalandhar and Tehsildar Sahib S. Manjit Singh, Jalandhar Registrar Shaheedi S. Manjit Singh, and the Gurudwara Saheeban will be started on Friday June 14 at 4 pm. The religious program of Sri Guru Granth Sahib Path ji, Jod Mela, sports are also scheduled. The religious program before the annual fair is being organized from June 12."
  },
  {
    "headline": "Senior Manager Baljit Singh appeals for strong support from the Sangat",
    "location": null,
    "summarised_content": "Senior Manager Baljit Singh appealed to the Sangat to cooperate in maintaining the discipline on the occasion of Jor Mela. He said that all arrangements have been made by the organizers. He said that all the devotees should participate in the Sangat and follow the organizers. He facilitates all the devotees so that the happiness of Baba Ji can be achieved."
  },
  {
    "headline": "Clears 3 crore houses for poor under PM Awas Yojana",
    "location": null,
    "summarised_content": "The government has been implementing Pradhan Mantri Awas Yojana since 2015-16 to provide basic amenities of houses with construction of toilets, tap connections, and stable and for rural houses, schemes for the eligible families under the housing for all (HFA) scheme were completed in the last 10 years under PMAY.  The basic amenities with the other provided schemes are in last 10 years provided such as functional household tap connection, functional toilet, electricity connection through convergence with governments and state governments."
  },
  {
    "headline": "SKM chief Tamang sworn in as Chief Minister of Sikkim",
    "location": "Sikkim",
    "summarised_content": "Sikkim Krantikari Morcha (SKM) president Prem Singh Tamang was sworn in as the Chief Minister of Sikkim. He was administered the oath of office and secrecy by Governor Lakshman Prasad Acharya at the Paljor Stadium, here. The Chief Minister became the Tamang, 56, became the second person to be sworn in as Chief Minister for a second time by a landslide victory, returning power to the Sikkim Krantikari Morcha (SKM) after a 33-year-old seats in 32-member state assembly. Sikkim Democratic Front (SDF), which ruled the state for 25 years, in a row since 1994, could manage only one seat in 2019. Could manage only one seat in 2019."
  },
  {
    "headline": "Jalandhar West Sibin C",
    "location": "Jalandhar",
    "summarised_content": "Punjab Chief Electoral Officer Sibin C has informed that voting for the Jalandhar West by-election will be held on July 10. The Election Commission of India has released the election program in this regard. Filing nominations will be held for Friday and the last date (Friday) for the last date will be on June 21. The nomination papers will be scrutinized on June 24 (Monday), while the last date (Wednesday) will be on June 26. Voting will be held on that day as per agreement."
  },
  {
    "headline": "The result will be announced on July 10 (Wednesday) and the counting of votes after the July 13 (Saturday).",
    "location": null,
    "summarised_content": "The result will be announced on July 10 (Wednesday) and the counting of votes after the July 13 (Saturday). The MCC will remain in Jalandhar from today. West constituency from applicable (Jalandhar) has been applicable. The MCC will remain in Jalandhar from today. West constituency from applicable (Jalandhar) has been applicable. The MCC will remain in Jalandhar from today. West constituency from applicable (Jalandhar) has been applicable."
  },
  {
    "headline": "The process will remain in force until the completion by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the MCC until the election process is completed by the